In [1]:
import sys
sys.path.append("/home/stachu/Projects/Anomaly_detection/Forecasting_models")

from predpy.dataset import TimeSeriesRecordsDataset
from predpy.dataset import SingleTimeSeriesDataset, MultiTimeSeriesDataset
from predpy.data_module import MultiTimeSeriesModule
from predpy.experimentator import (
    DatasetParams, ModelParams, ExperimentatorPlot,
    Experimentator, load_experimentator, plot_aggregated_predictions)
from predpy.preprocessing import set_index
from predpy.preprocessing import moving_average
from predpy.preprocessing import (
    load_and_preprocess, set_index, moving_average, drop_if_is_in,
    use_dataframe_func, loc, iloc)
from predpy.trainer import (
    CheckpointParams, TrainerParams, EarlyStoppingParams, LoggerParams)

import pickle
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tsai.models import TCN, ResNet, TST, RNN, TransformerModel, FCN
import pandas as pd
from pytorch_lightning.loggers import TensorBoardLogger

In [2]:
window_size = 366

load_params = {
    "sep": ';', "header": 0, "low_memory": False,
    "infer_datetime_format": True, "parse_dates": {'datetime': [0, 1]},
    "index_col": ['datetime']
}

columns = ["Global_active_power", "Voltage"]
drop_refill_pipeline = [
    (loc, {"columns": columns}),
    (drop_if_is_in, (["?", np.nan]), {"columns": columns}),
    (iloc, {"rows_end": 1500}),
    # (iloc, {"rows_start": -20000}),
]
preprocessing_pipeline = [
    (use_dataframe_func, "astype", "float"),
]
datasets_params = [
    DatasetParams(
        path="/home/stachu/Projects/Anomaly_detection/Forecasting_models/data/Energy/household_power_consumption/household_power_consumption.csv",
        load_params=load_params,
        target="Global_active_power",
        split_proportions=[0.8, 0.1, 0.1],
        window_size=window_size,
        batch_size=64,
        DatasetCls=MultiTimeSeriesDataset,
        drop_refill_pipeline=drop_refill_pipeline,
        preprocessing_pipeline=preprocessing_pipeline,
        scaler=MinMaxScaler()),
]

c_in = 2
c_out = 1

models_params = [
    # ModelParams(
    #     name_="TST_l3_fcDrop0.1", cls_=TST.TST,
    #     init_params={
    #         "c_in": c_in, "c_out": c_out, "seq_len": window_size,
    #         "max_seq_len": window_size, "n_layers": 3, "fc_dropout": 0.1}),
    # ModelParams(
    #     name_="TST_l2_fcDrop0.1", cls_=TST.TST,
    #     init_params={
    #         "c_in": c_in, "c_out": c_out, "seq_len": window_size,
    #         "max_seq_len": window_size, "n_layers": 2, "fc_dropout": 0.1}),
    # ModelParams(
    #     name_="TST_l2_fcDrop0.0", cls_=TST.TST,
    #     init_params={
    #         "c_in": c_in, "c_out": c_out, "seq_len": window_size,
    #         "max_seq_len": window_size, "n_layers": 2, "fc_dropout": 0.0}),
    # ModelParams(
    #     name_="ResNet", cls_=ResNet.ResNet,
    #     init_params={"c_in": c_in, "c_out": c_out}),
    ModelParams(
        name_="LSTM_h200_l1", cls_=RNN.LSTM,
        init_params={
            "c_in": c_in, "c_out": c_out, "hidden_size": 200, "n_layers": 1}),
    # ModelParams(
    #     name_="LSTM_h200_l2", cls_=RNN.LSTM,
    #     init_params={
    #         "c_in": c_in, "c_out": c_out, "hidden_size": 200, "n_layers": 2}),
    # ModelParams(
    #     name_="LSTM_h400_l1", cls_=RNN.LSTM,
    #     init_params={
    #         "c_in": c_in, "c_out": c_out, "hidden_size": 400, "n_layers": 1}),
]

chp_p = CheckpointParams(
    dirpath="../checkpoints", monitor='val_loss', verbose=True,
    save_top_k=1)
tr_p = TrainerParams(
    max_epochs=1, gpus=1, auto_lr_find=True,
    logger=TensorBoardLogger("../lightning_logs"))
es_p = EarlyStoppingParams(
    monitor='val_loss', patience=2, verbose=True)

In [3]:
# exp = Experimentator(
#     models_params=models_params,
#     datasets_params=datasets_params,
#     trainer_params=tr_p,
#     checkpoint_params=chp_p,
#     early_stopping_params=es_p
# )

# exp.run_experiments(experiments_path="../saved_experiments", safe=False)

exp = load_experimentator(
    "../saved_experiments/2021-12-10_11:54:55.pkl")

Global seed set to 42
/home/stachu/anaconda3/envs/dl/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning:

Checkpoint directory ../checkpoints/household_power_consumption/LSTM_h200_l1 exists and is not empty.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/stachu/anaconda3/envs/dl/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning:

DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | model     | LSTM    | 163 K 
1 | criterion | MSELoss | 0     
--------------------------------------
163 K     Trainable params
0         Non-trainable params
163 K     Total params
0.654     Total estimated model params size (MB)


Global seed set to 42


Epoch 0:   0%|          | 0/7 [00:00<?, ?it/s] 

/home/stachu/anaconda3/envs/dl/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:326: UserWarning:

The number of training samples (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.



Epoch 0: 100%|██████████| 7/7 [00:00<00:00, 11.45it/s, loss=0.101, v_num=3:31, train_loss=0.173, val_loss=0.137]

Metric val_loss improved. New best score: 0.137
Epoch 0, global step 5: val_loss reached 0.13665 (best 0.13665), saving model to "/home/stachu/Projects/Anomaly_detection/Forecasting_models/checkpoints/household_power_consumption/LSTM_h200_l1/2021-12-10_17:43:31.ckpt" as top 1


Epoch 0: 100%|██████████| 7/7 [00:00<00:00, 10.98it/s, loss=0.101, v_num=3:31, train_loss=0.173, val_loss=0.137]

Making predictions:   0%|          | 0/1 [00:00<?, ?it/s]

Making predictions: 100%|██████████| 1/1 [00:00<00:00,  3.93it/s]


In [4]:
tsm = exp.load_time_series_module(0)

# anomalies = []
# i = 0

# for data in tsm.train_dataloader():
#     anomalies += [data]

In [5]:
from typing import Union, Tuple, Literal, Callable, List, Type
from torch.utils.data import Dataset, DataLoader
from functools import wraps


def copy_data_from_range(
    tsm: MultiTimeSeriesModule,
    range_: Union[
        Tuple[int, int],
        Literal["all", "train", "val", "test"]] = "all"
) -> List[pd.DataFrame]:
    return tsm.sequences


def seqs_to_type(
    seqs: List[pd.DataFrame],
    type_: Union[
        MultiTimeSeriesModule, DataLoader, Dataset, List[pd.DataFrame]]
) -> Union[MultiTimeSeriesModule, DataLoader, Dataset, List[pd.DataFrame]]:
    return seqs


# def anomaly_creation_wrapper(anomaly_creation):
#     @wraps
#     def create_anomaly(
#         tsm: MultiTimeSeriesModule,
#         range_: Union[
#             Tuple[int, int],
#             Literal["all", "train", "val", "test"]] = "all",
#         deal_with_negativity: str = None,
#         return_type: Union[
#             MultiTimeSeriesModule, DataLoader, Dataset,
#             List[pd.DataFrame]] = None,
#         *args, **kwargs
#     ):
#         seqs_dfs = copy_data_from_range(tsm, range_)
#         seqs_dfs = anomaly_creation(
#             seqs_dfs, deal_with_negativity, *ars, **kwargs)
#         result = seqs_to_type(seqs_dfs, type_=return_type)
#         return result
#     return create_anomaly


def create_anomaly(
    tsm: MultiTimeSeriesModule,
    anomaly_creation: Callable,
    range_: Union[
        Tuple[int, int],
        Literal["all", "train", "val", "test"]
        ] = "all",
    deal_with_negativity: str = None,
    return_type: Union[
        Type[MultiTimeSeriesModule], Type[DataLoader], Type[Dataset],
        Type[List[pd.DataFrame]]
        ] = None,
    *args, **kwargs
):
    seqs_dfs = copy_data_from_range(tsm, range_)
    seqs_dfs = anomaly_creation(
        seqs=seqs_dfs,
        deal_with_negativity=deal_with_negativity,
        *args, **kwargs)
    result = seqs_to_type(seqs_dfs, type_=return_type)
    return result

def add_nothing(
    seqs: List[pd.DataFrame],
    deal_with_negativity: str = None,
):
    return seqs

create_anomaly(
    tsm=tsm, anomaly_creation=add_nothing, range_="all",
    deal_with_negativity="abs", return_type=List[pd.DataFrame]
)

# def add_white_noise(
#     tsm: MultiTimeSeriesModule,
#     range_: Union[
#         Tuple[int, int],
#         Literal["all", "train", "val", "test"]] = "all",
#     deal_with_negativity: str = None,
#     return_type: Union[
#         MultiTimeSeriesModule, DataLoader, Dataset,
#         List[pd.DataFrame]] = None
# ):
#     # wczytywanie danych z zakresu

#     for seq in tsm.sequences:
#         print(seq)
#         if positive:
#             print(seq.apply(lambda x: abs(x + np.random.randn())))
#         else:
#             print(seq.apply(lambda x: x + np.random.randn()))
#         break

#     # zwracanie w odpowiedniej postaci
# add_white_noise(tsm, positive=True)

[                     Global_active_power   Voltage
 datetime                                          
 2006-12-16 17:24:00             0.534667  0.209897
 2006-12-16 17:25:00             0.687200  0.144100
 2006-12-16 17:26:00             0.689067  0.125612
 2006-12-16 17:27:00             0.690933  0.150082
 2006-12-16 17:28:00             0.461333  0.255574
 ...                                  ...       ...
 2006-12-17 18:19:00             0.577333  0.182708
 2006-12-17 18:20:00             0.453333  0.203915
 2006-12-17 18:21:00             0.451733  0.195759
 2006-12-17 18:22:00             0.454400  0.243611
 2006-12-17 18:23:00             0.454133  0.243067
 
 [1500 rows x 2 columns]]

In [ ]:
from tsad.anomaly_detector import PredictionAnomalyDetector

model = exp.load_pl_model(
    model_idx=0,
    dir_path="checkpoints/household_power_consumption/LSTM_h200_l1")

ad = PredictionAnomalyDetector(model)

ad.fit(
    dataloader=tsm.train_dataloader(),
    anomaly_data=tsm.val_dataloader(),
    normal_data=tsm.test_dataloader()
)

ad.find_anomalies(tsm.test_dataloader())

In [ ]:
# BTC = yf.Ticker("BTC-USD")
# # df = BTC.history(period="max", interval="1d", auto_adjust=False, actions=False)
# df = BTC.history(interval="1d", start="2014-09-17", end="2021-10-16", actions=False)
# df["Open-Close-diff"] = df.apply(lambda x: x["Close"] - x["Open"], axis=1)

# from datetime import datetime
# end = datetime.strptime("06-01-2017", '%m-%d-%Y')
# df = df.loc[:end]  # remove last years due to a change behaviour
# df = df[["Close", "Open-Close-diff"]]